In [1]:
from pathlib import Path

import geopandas
import numpy
import pandas
import snail.damages
import snail.intersection
import snail.io
import snkit

from pyproj import Geod
from scipy.integrate import simpson
from tqdm.auto import tqdm

In [3]:
# set up variables for incoming and processed data paths
project_path = Path().resolve().parent # assume we're running from the project scripts directory, so get the parent
incoming_data_path = project_path / "incoming_data"
processed_data_path = project_path / "processed_data"

In [4]:
hazard_paths = list((incoming_data_path / "starter-data-kit" / "data" / "COD" / "jrc_floods").glob("*.tif"))
hazard_files = pandas.DataFrame({"path": hazard_paths})
hazard_files["key"] = [Path(path).stem for path in hazard_paths]
hazard_files.key = hazard_files.key.str.replace("floodMapGL_", "").str.replace("y__COD", "")
hazard_files.key = "jrc_flood_" + hazard_files.key + "_depth_m"
hazard_files, grids = snail.io.extend_rasters_metadata(hazard_files)
assert len(grids) == 1
grid = grids[0]
hazard_files

,path,key,grid_id,bands
0,/home/mert2014/projects/snail-datapkg-demos/co...,jrc_flood_rp50_depth_m,0,"(1,)"
1,/home/mert2014/projects/snail-datapkg-demos/co...,jrc_flood_rp100_depth_m,0,"(1,)"
2,/home/mert2014/projects/snail-datapkg-demos/co...,jrc_flood_rp10_depth_m,0,"(1,)"
3,/home/mert2014/projects/snail-datapkg-demos/co...,jrc_flood_rp20_depth_m,0,"(1,)"
4,/home/mert2014/projects/snail-datapkg-demos/co...,jrc_flood_rp200_depth_m,0,"(1,)"
5,/home/mert2014/projects/snail-datapkg-demos/co...,jrc_flood_rp500_depth_m,0,"(1,)"


In [5]:
network_path = processed_data_path / "networks" / "transport" / "road.2025-04-30.gpkg"
network = snkit.network.read_file(network_path, nodes_layer="road_nodes", edges_layer="road_edges")

In [6]:
prepared = snail.intersection.prepare_linestrings(network.edges)

flood_intersections = snail.intersection.split_linestrings(prepared, grid)

# push into split_linestrings
flood_intersections = snail.intersection.apply_indices(
    flood_intersections, grid, index_i="i_0", index_j="j_0"
)

flood_intersections = snail.io.associate_raster_files(
    flood_intersections, hazard_files
)

# calculate the length of each line
geod = Geod(ellps="WGS84")
flood_intersections["length_m"] = flood_intersections.geometry.apply(
    geod.geometry_length
)

In [7]:
# clip negative flood depths to zero
for colname in flood_intersections.columns:
    if "jrc_flood" in colname:
        flood_intersections[colname] = flood_intersections[colname].clip(0, 25)

In [8]:
damage_curves = {}
for damage_col in ["paved","unpaved"]:
    damage_curve = snail.damages.PiecewiseLinearDamageCurve.from_csv(
        incoming_data_path / "nirandjan-2024-vulnerabilty/road_damage.csv",
        intensity_col="depth_m",
        damage_col=damage_col
    )
    damage_curves[damage_col.split("_")[0]] = damage_curve
damage_curves.keys()

dict_keys(['paved', 'unpaved'])

In [33]:
dfs = []
for paved_or_unpaved, group in flood_intersections.groupby("paved"):
    damage_curve = damage_curves[paved_or_unpaved]

    for colname in flood_intersections.columns:
        if "depth_m" in colname:
            damage_col = colname.replace("depth_m", "damage_fraction")
            damage_fraction = damage_curve.damage_fraction(group[colname])
            group[damage_col] = damage_fraction
    dfs.append(group)
damage_costs = pandas.concat(dfs)
damage_costs

,osm_id,name,ref,highway,lanes,paved,id,from_id,to_id,length_m,...,jrc_flood_rp10_depth_m,jrc_flood_rp20_depth_m,jrc_flood_rp200_depth_m,jrc_flood_rp500_depth_m,jrc_flood_rp50_damage_fraction,jrc_flood_rp100_damage_fraction,jrc_flood_rp10_damage_fraction,jrc_flood_rp20_damage_fraction,jrc_flood_rp200_damage_fraction,jrc_flood_rp500_damage_fraction
0,4392435,Boulevard Sendwe,None,trunk,3,paved,edge_0,node_2,node_3,31.443020,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,4392435,Boulevard Sendwe,None,trunk,3,paved,edge_1,node_3,node_4,112.962245,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,4392435,Boulevard Sendwe,None,trunk,3,paved,edge_2,node_4,node_5,18.376844,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4392435,Boulevard Sendwe,None,trunk,3,paved,edge_3,node_5,node_6,58.293174,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4392435,Boulevard Sendwe,None,trunk,3,paved,edge_4,node_6,node_7,95.743190,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1103224,1376478716,None,None,tertiary,2,unpaved,edge_1103224,node_1100878,node_1100879,99.297208,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1103225,1376478716,None,None,tertiary,2,unpaved,edge_1103225,node_1100879,node_1100880,74.814206,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1103226,1376478716,None,None,tertiary,2,unpaved,edge_1103226,node_1100880,node_1100881,83.281072,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1103227,1376478716,None,None,tertiary,2,unpaved,edge_1103227,node_1100881,node_523681,30.354358,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [35]:
# highly approximate cost assumptions for rehabilitation per
# lane-kilometre, drawn from Koks et al 2019 and rounded
cost_assumptions = {
    "paved": 500_000,
    "unpaved": 25_000,
}
damage_costs["cost_usd_per_km_per_lane"] = damage_costs.paved.map(cost_assumptions)

# Other global/regional reference sources include for example those cited in
# https://github.com/nismod/open-gira/blob/v0.3.2/config/rehab_costs/road.csv

In [ ]:
damage_costs.lanes = damage_costs.lanes.astype(int)
damage_costs.lanes.unique()

damage_costs['rehab_cost_usd'] = (
    damage_costs.cost_usd_per_km_per_lane
    * damage_costs.lanes
    * damage_costs.length_m
    / 1000
)

In [ ]:
for colname in damage_costs.columns:
    if "damage_fraction" in colname:
        damage_col = colname.replace("damage_fraction", "damage_cost")
        damage_cost = (
            damage_costs[colname]
            * damage_costs.cost_usd_per_km_per_lane
            * damage_costs.lanes
            * damage_costs.length_m
            / 1000
        )
        damage_costs[damage_col] = damage_cost
damage_costs

,osm_id,name,ref,highway,lanes,paved,id,from_id,to_id,length_m,...,jrc_flood_rp200_damage_fraction,jrc_flood_rp500_damage_fraction,cost_usd_per_km_per_lane,rehab_cost_usd,jrc_flood_rp50_damage_cost,jrc_flood_rp100_damage_cost,jrc_flood_rp10_damage_cost,jrc_flood_rp20_damage_cost,jrc_flood_rp200_damage_cost,jrc_flood_rp500_damage_cost
0,4392435,Boulevard Sendwe,None,trunk,3,paved,edge_0,node_2,node_3,31.443020,...,0.0,0.0,500000,47164.529747,0.0,0.0,0.0,0.0,0.0,0.0
1,4392435,Boulevard Sendwe,None,trunk,3,paved,edge_1,node_3,node_4,112.962245,...,0.0,0.0,500000,169443.368191,0.0,0.0,0.0,0.0,0.0,0.0
2,4392435,Boulevard Sendwe,None,trunk,3,paved,edge_2,node_4,node_5,18.376844,...,0.0,0.0,500000,27565.265723,0.0,0.0,0.0,0.0,0.0,0.0
3,4392435,Boulevard Sendwe,None,trunk,3,paved,edge_3,node_5,node_6,58.293174,...,0.0,0.0,500000,87439.760724,0.0,0.0,0.0,0.0,0.0,0.0
4,4392435,Boulevard Sendwe,None,trunk,3,paved,edge_4,node_6,node_7,95.743190,...,0.0,0.0,500000,143614.784328,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1103224,1376478716,None,None,tertiary,2,unpaved,edge_1103224,node_1100878,node_1100879,99.297208,...,0.0,0.0,25000,4964.860399,0.0,0.0,0.0,0.0,0.0,0.0
1103225,1376478716,None,None,tertiary,2,unpaved,edge_1103225,node_1100879,node_1100880,74.814206,...,0.0,0.0,25000,3740.710289,0.0,0.0,0.0,0.0,0.0,0.0
1103226,1376478716,None,None,tertiary,2,unpaved,edge_1103226,node_1100880,node_1100881,83.281072,...,0.0,0.0,25000,4164.053609,0.0,0.0,0.0,0.0,0.0,0.0
1103227,1376478716,None,None,tertiary,2,unpaved,edge_1103227,node_1100881,node_523681,30.354358,...,0.0,0.0,25000,1517.717876,0.0,0.0,0.0,0.0,0.0,0.0


In [39]:
(project_path / "results").mkdir(exist_ok=True)
damage_costs.to_parquet(project_path / "results" / "split_road_damages.gpq")

In [48]:
rp_cols = [c for c in damage_costs.columns if "damage_cost" in c]
def get_rp(col):
    return int(col.replace("jrc_flood_rp", "").replace("_damage_cost", ""))

rp_cols = sorted(rp_cols, key=lambda col: 1 / get_rp(col))
rps = numpy.array([get_rp(col) for col in rp_cols])
probabilities = 1 / rps
rp_damages = damage_costs[rp_cols]
ead = simpson(rp_damages, x=probabilities, axis=1)
damage_costs["jrc_flood_ead"] = ead

# write to CSV
split_damages = damage_costs[
    ["osm_id","name","highway","lanes","paved","id","from_id", "to_id"]
    + rp_cols
    + ["jrc_flood_ead"]
]
joined_damages = split_damages.groupby(
    ["osm_id","name","highway","lanes","paved","id","from_id", "to_id"]
).sum().query("jrc_flood_ead > 0")

joined_damages.to_csv(project_path / "results" / "road_damages.csv")

In [47]:
all_edges_with_damages = (
    network.edges
    .set_index("id")
    .join(
        joined_damages.reset_index()
        [rp_cols + ["jrc_flood_ead", "id"]]
        .set_index("id")
    )
    .fillna({col: 0 for col in rp_cols})
    .fillna({"jrc_flood_ead": 0})
)

In [49]:
all_edges_with_damages.to_file(project_path / "results" / "road_damages.gpkg")